<a href="https://colab.research.google.com/github/DhruPithadia/DhruPithadia-DataScience-GenAI-repo/blob/main/Assignment_2/2_02_Joining_DataFrames_COMPLETED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 2.02 Pandas: Joining and merging dataframes
### Why Combine Dataframes?
Quite often in practical applications we have data coming from multiple sources such as different files, database tables or IoT devices. This is particularly likely when working in data science and artificial intelligence. However, typically we want to create a single dataframe to run our analysis and/or models. To achieve this commonly we would want to combine dataframes together.

Joining data is a large topic and an important one in database administration and data engineering. We will look at this at a relatively high level, althout the interested reader can certainly explore this in much more depth.

Let us begin by using the orders_df from the previous session.

In [1]:
import pandas as pd
import numpy as np

orders = {'o10001':{'date':'2024/01/10', 'product':'Hoodie', 'quantity':'1'},
            'o10002':{'date':'2024/01/13', 'product':'Tote bag', 'quantity':'2'},
            'o10003':{'date':'2024/01/14', 'product':'Pencil', 'quantity':'10'},
            'o10004':{'date':'2024/01/15', 'product':'T-shirt', 'quantity':'2'}
}

orders_df = pd.DataFrame(orders)
orders_df

,o10001,o10002,o10003,o10004
date,2024/01/10,2024/01/13,2024/01/14,2024/01/15
product,Hoodie,Tote bag,Pencil,T-shirt
quantity,1,2,10,2


In this scenario, we also have some data about these products from our operations team. The data is currently in a dictionary and can be transformed in the usual fashion:

In [2]:
products = {'123':{'name':'Hoodie', 'cost_price':12.12, 'sale_price':15.00},
            '124':{'name':'Tote bag', 'cost_price':2.15, 'sale_price':9.99},
            '125':{'name':'Pencil', 'cost_price':22.45, 'sale_price':49.99},
            '126':{'name':'T-shirt', 'cost_price':0.45, 'sale_price':2.99},
            '127':{'name':'Honours degree', 'cost_price':0.78, 'sale_price':1.49}
}

products_df = pd.DataFrame(products)
products_df

,123,124,125,126,127
name,Hoodie,Tote bag,Pencil,T-shirt,Honours degree
cost_price,12.12,2.15,22.45,0.45,0.78
sale_price,15.0,9.99,49.99,2.99,1.49


### Joining Dataframes using Merge
What we want to do is join these datasets together. However, currently the Dataframes are both the wrong way round. We would want our rows in orders_df to be the order ID, and the rows in products_df to be the product ID. We can transform the data using the Dataframe.transpose function:

In [3]:
orders_df = orders_df.transpose()
products_df = products_df.transpose()

Transpose basically flips the axes of the Dataframe so the current columns become rows and vice versa. Note, _Dataframe.transpose_ creates a new object by default. We use the "orders_df = ..." element at the start to reassign the variable and make orders_df equal the new, transposed version.

In [4]:
orders_df

,date,product,quantity
o10001,2024/01/10,Hoodie,1
o10002,2024/01/13,Tote bag,2
o10003,2024/01/14,Pencil,10
o10004,2024/01/15,T-shirt,2


With this in place we can join the two Dataframes. There are several decisions that need to be taken when joining two datasets. Firstly, there are multiple ways to join data in pandas but the most appropriate here would seem to be using the _merge( )_ function. These choices will be discussed in more detail later in the notebook. Secondly we need to decide on the element that we will make the join on. In this case, the data that the two dataframes have in common are the product name, labelled “product” in orders_df and as “name” in products_df. We also need to decide the type of join – what do we want to do with rows that are in one set but not in the other? This too is discussed further below. In this case the scenario is to get more details on our orders, so we are joining products_df to orders_df so we will base our rows on the orders_df dataframe. With these choices in mind we get the following code:

In [5]:
joined_df = orders_df.merge(products_df, how='left', left_on='product', right_on='name')

We’ll unpack this a little more because it is a more complex function. We are creating a new dataframe called joined_df. This will be based on the orders_df being merged with products_df as a ‘left join’. This means we will use the rows of the orders_df not products_df (as orders_df is on the left hand of the command). Finally we need to tell Python which columns we will be joining based on. In this case the left hand dataframe (orders_df) lists the relevant data as “product”, and the right hand (products_df) lists it as “name”.

In [6]:
joined_df

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


There are several types of joins that can be used here. The main types are shown below. The yellow bit in each Venn diagram represents the bit we would keep if we perform this type of join.

So far we have only looked at a "left join" (labelled in pandas as "left"). It would also be possible to specify an "inner join" ("inner"). This means we only keep records (rows) that are in both datasets. Only the records in both would be included if we use an "inner join". Finally we could use a "full join" ("outer") which means that we keep all the records irrespective of whether they are missing in one of the sides.

![](https://drive.google.com/uc?export=view&id=1e8rPwaCqpnrB9-yIcqKOm2sb6VUPGz_g)

Let’s return to our first merged dataframe (joined_df) as this better fits the description of this problem (we care more about orders in this problem than products). One immediate issue is that we have duplicated columns product and name. Let’s tidy this up by dropping the name column:

In [7]:
joined_df = joined_df.drop(['name'], axis=1)
joined_df

,date,product,quantity,cost_price,sale_price
0,2024/01/10,Hoodie,1,12.12,15.0
1,2024/01/13,Tote bag,2,2.15,9.99
2,2024/01/14,Pencil,10,22.45,49.99
3,2024/01/15,T-shirt,2,0.45,2.99


The _drop( )_ function can be used to remove columns or rows. axis=1 indicates that we want to a drop a column(s), axis=0 would mean we were dropping a row(s). Note as well that although we are only dropping one column we still need to present the column name inside a list as this is what the function expects (a list).

### EXERCISE
In the above example we merged using a left join. How would different types of joins effect the final dataframe? Experiment with inner join, right join and outer join. What impact does it have on the data retained? Are the results what you expected them to be?

# EXERCISE TASK

## Experimenting with Different Join Types

To understand the impact of various join types (inner, right, and outer) on data retention and `NaN` values, I will perform each type of merge operation between `orders_df` and `products_df`. I'll use 'product' from `orders_df` and 'name' from `products_df` as the keys for merging. For each join, I will display the resulting DataFrame and provide my observations, relating them back to the theoretical behavior of joins, including the left join we explored earlier.

### Inner Join

**What this code will do:** The following code will perform an **inner join** on `orders_df` and `products_df`. This type of join keeps only the rows where the joining keys ('product' in `orders_df` and 'name' in `products_df`) have an exact match in *both* DataFrames.

**Why this is useful:** An inner join is ideal when you want to focus solely on the common elements or records that are present in both datasets. It ensures that the resulting DataFrame contains only fully matched information, avoiding any `NaN` values due to unmatched entries.

In [11]:
inner_joined_df = orders_df.merge(products_df, how='inner', left_on='product', right_on='name')
inner_joined_df

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


### My Observations on the Inner Join

When I performed the inner join between `orders_df` and `products_df` using 'product' and 'name' as the matching keys, the resulting dataframe, `inner_joined_df`, only contained rows where there was a perfect match in both dataframes.

Looking at the output, I saw four rows: 'Hoodie', 'Tote bag', 'Pencil', and 'T-shirt'. All these products exist in both my `orders_df` and `products_df`. Interestingly, the 'Honours degree' from `products_df` wasn't included because it didn't have a corresponding order.

What I noticed is that this join type didn't introduce any `NaN` (Not a Number) values due to unmatched entries, which makes sense because it only keeps records that are fully present in both sets. This really aligns with the idea of an inner join keeping only the overlapping part of the Venn diagram.

### Right Join

**What this code will do:** This code will execute a **right join** operation. A right join includes all rows from the 'right' DataFrame (`products_df`) and only the matching rows from the 'left' DataFrame (`orders_df`). If a row from `products_df` does not have a match in `orders_df`, the columns originating from `orders_df` will be filled with `NaN` values.

**Why this is useful:** A right join is particularly useful when you want to ensure that all records from a secondary dataset (the 'right' one) are preserved, and then see which of those records have corresponding information in your primary dataset. This allows you to identify items in your secondary list that might be missing links to the primary one.

In [12]:
right_joined_df = orders_df.merge(products_df, how='right', left_on='product', right_on='name')
right_joined_df

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99
4,NaN,NaN,NaN,Honours degree,0.78,1.49


### My Observations on the Right Join

Next, I experimented with the right join. This time, when I merged `orders_df` with `products_df` using `how='right'`, the `right_joined_df` included all entries from `products_df` (my 'right' dataframe).

I observed five rows in total. The four products that were also in `orders_df` ('Hoodie', 'Tote bag', 'Pencil', 'T-shirt') had all their details from both dataframes. However, for 'Honours degree', which was only in `products_df`, the columns that would normally come from `orders_df` ('date', 'product', and 'quantity') were filled with `NaN` values. This showed me that the right join makes sure to keep everything from the right side, even if it means filling in blanks for the left side.

It also confirmed that if there were any orders in `orders_df` that didn't have a matching product, those wouldn't show up here, because the right join prioritizes the right dataframe. This is exactly what the Venn diagram for a right join illustrates – the whole right circle, plus the overlap.

### Outer Join

**What this code will do:** This section performs an **outer join** between `orders_df` and `products_df`. An outer join is the most inclusive type, retaining all rows from *both* DataFrames. If a row from one DataFrame doesn't have a match in the other, the corresponding columns from the non-matching DataFrame will be populated with `NaN` values.

**Why this is useful:** An outer join is invaluable when you need a complete picture of all records across both datasets, including all matches and all non-matches. It helps in identifying data gaps or inconsistencies, showing you where information might be missing from either side.

In [13]:
outer_joined_df = orders_df.merge(products_df, how='outer', left_on='product', right_on='name')
outer_joined_df

,date,product,quantity,name,cost_price,sale_price
0,NaN,NaN,NaN,Honours degree,0.78,1.49
1,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99
4,2024/01/13,Tote bag,2,Tote bag,2.15,9.99


### My Observations on the Outer Join

Finally, I tried the outer join. With `how='outer'`, the `outer_joined_df` aimed to include all rows from *both* `orders_df` and `products_df`.

I saw five rows again. The four matching products had all their data. The key difference was how it handled the unmatched 'Honours degree'. Just like with the right join, the columns from `orders_df` ('date', 'product', 'quantity') for 'Honours degree' were `NaN`. If I had any orders for products not in `products_df`, those would also show up, but with `NaN`s for the product details.

This really drove home the point that an outer join gives you the complete picture, showing you everything, including where data is missing from either side. It's like combining both circles in a Venn diagram, with empty spots where information doesn't exist in one of the original datasets.

## Summary of experimenting with different join types

After experimenting with different join types, I have a much clearer understanding of how they affect data retention and the presence of `NaN` values. Here's a summary of the key learnings and practical implications of what I observed:

*   Choosing the right join type is critical for data integrity and analysis goals. Each type serves a distinct purpose.
*   An inner join focuses on common elements, useful when you only care about records that exist in both datasets.
*   Left/right joins prioritize one dataset while incorporating data from the other, ideal for preserving a primary list and adding supplementary details.
*   An outer join provides a complete, comprehensive view, highlighting all matches and non-matches, and is excellent for identifying data gaps across both datasets.
*   Understanding how `NaN` values are generated during different join operations is crucial for subsequent data cleaning and accurate downstream analysis, as they directly indicate missing or unmatched information.